# Assignment 1

This exercise is meant to help you get familiar with some language data. 

The corpus used is the **Penn Treebank**, which is a collection of data from the newspaper 
The Wall Street Journal. The exercise will not take more than a few lines of code; the idea is to examine the data, and notice interesting properties.

## Required software

### Easy installation

The easiest way to get the required software is to install Anaconda. See https://www.continuum.io/downloads . It contains all required packages, including python and jupyter. You can choose python 2.7 or 3.5.

### Manual installation

Make sure that you have `numpy` and `matplotlib` installed. If you don't, you can use e.g. `pip install <package> --user` (python2) or `pip3 install <package> --user` (python3).

## Submission
We will grade your assignments with pass/fail/good, so don't forget to hand them in! 
Choose `File->Download as->HTML` and check if the HTML-file contains all your answers. You can work and submit in pairs. Please e-mail your TA with `"[NLP1] Assignment 1"` as the subject. 
**The deadline for submission is Sunday 6 nov 23:59.**

## Start the notebook

Start a terminal, and `cd` into the directory where you saved the notebook. Then type `jupyter notebook`. Your web browser will open.

## Exercise 1.1

You are provided with a corpus containing words and their Part-of-speech tags in the format is
**word|POS** (one sentence per line) (file name : **sec02-21.gold.tagged**). This data is extracted from Sections 02-21 from the Penn Treebank: these sections are most commonly used for training statistical models (like POS-taggers and parsers).

(a) What are the total number of words (tokens) in this corpus? 
What is the number of distinct word types?


In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
import numpy as np
import operator
import functools

In [ ]:
def read_corpus(data):
    n_sentences = 0
    
    word_pos_combo = []
    word_pos_dic = dict()
    words = []
    pos = []

    # Read text from file
    with open(data, 'r') as f:
        for sentence in f:
            n_sentences += 1
            temp = sentence.split()
            
            for i in range(len(temp)):
                token = temp[i].split('|')
                
                word_pos_combo.append(temp[i])
                words.append(token[0])
                pos.append(token[1])
                
                if token[0] in word_pos_dic:
                    # link word to new POS seen
                    if token[1] not in word_pos_dic[token[0]]:
                        word_pos_dic[token[0]].append(token[1])
                else:
                    # create new entry for word
                    word_pos_dic[token[0]] = [token[1]]
                            
            
    # Get only distinct words/pos
    distinct_word_pos_combo = set(word_pos_combo)
    distinct_words = set(words)
    distinct_pos = set(pos)
    
    # Display dataset information
    print "Number of sentences in ", data, ":", n_sentences
    print "Number of words in ", data, ":", len(words)
    
    print "\nNumber of distinct words in", data, ":", len(distinct_words)
    print "Number of distinct pos in", data, ":", len(distinct_pos)
    print "Number of distinct word-pos combinations in ", data, ":", len(distinct_word_pos_combo)
    
    return words, pos, word_pos_combo, word_pos_dic, distinct_words, distinct_pos, distinct_word_pos_combo

# Proccess training data
words, pos, word_pos_combo, word_pos_dic, distinct_words, distinct_pos, distinct_word_pos_combo = read_corpus('sec02-21.gold.tagged')

# We need the "clean" sentences in order to measure probabilities per sentence
clear_sentences = [] 
clear_text = (' ').join(words)
clear_sentences += clear_text.split(' . ')

In [ ]:
print word_pos_dic.keys()[1000], word_pos_dic[word_pos_dic.keys()[1000]]

(b) Plot a graph of word frequency versus rank of a word, in this corpus. Does this corpus obey Zipf’s law?

In [ ]:
corpus_words = Counter(words)
sorted_corpus_words = corpus_words.most_common(300)

# split words and counts
tokens = [x[0] for x in sorted_corpus_words]
counts = [x[1] for x in sorted_corpus_words]

# create plot
fig = plt.figure(figsize=(15,5))

# Plot bar with values from dict and label with keys
plt.bar(range(len(sorted_corpus_words)), counts, width=0.3)
plt.xticks(range(len(sorted_corpus_words)), tokens)

# Rotate labels by 90 degrees so you can see them
locs, tokens = plt.xticks()
plt.setp(tokens, rotation=90,fontsize=6)

fig.show()

Due to performance issues, we only take the first 3000 words for the plot. Howeveer, it is enough to show that the data follows Zipf's law, with a very long tail

(c) What is the 25th most common word in the corpus? How many times does it occur? What about the 50th most common word, the 100th and the 1000th?

In [ ]:
most_common_25 = corpus_words.most_common(25)[-1]
most_common_50 = corpus_words.most_common(50)[-1]
most_common_100 = corpus_words.most_common(100)[-1]

print "The 25th most common word: \n", most_common_25
print "\nThe 50th most common word: \n", most_common_50
print "\nThe 100th most common word: \n", most_common_100

(d) How many different Part-of-speech tags are present in the corpus?

In [ ]:
print "Number of distinct pos: ", len(distinct_pos)

(e) Print a list of the 10 most common part-of-speech tags. Spend a few minutes trying to guess what each tag means, by looking at associated words.

In [ ]:
corpus_pos = Counter(pos)

most_common_10 = corpus_pos.most_common(10)

print "The 10 most common pos: \n", most_common_10

(f) Assume that the probability $P(w_1^n)$ of a sentence $w_1 \ldots w_n$   can be calculated as follows:

$$P(w_1^n) = P(w_1) \cdot P(w_2) \ldots P(w_n) $$

The probability of a word $w_i$ can be calculated from a corpus as 
$$P(w_i) = \frac{count (w_i)}{N}$$ where $N$ is the total number of word tokens in the corpus. 

What is the probability of the first two sentences in the corpus? 

In [ ]:
#Function to measure the probability of a uni-gram (w) in a corpus of total count N
def get_word_probability(corpus, w):
    prob_w = (100.0 * float(corpus[w])) / (100. * float(len(corpus)))
    return prob_w


#Function to measure the probabilities of each word in a sentence and 
# the total probability of an n-gram sentence (s) in the corpus
def get_sentence_probability(corpus,sentence):
    #Probabilities of a sentence
    probs_of_s = []
    for word in sentence.split(" "):
        probs_of_s.append(get_word_probability(corpus,word))
    
    total_prob_s = functools.reduce(operator.mul, probs_of_s, 1)
    
    return probs_of_s, total_prob_s



# Probability of the first two sentences in the corpus
enum = 0
for sentence in clear_sentences:
    if enum < 2:
        print "Sentence is : ", clear_sentences[enum]
        print "Length of sentence is : ", len(clear_sentences[enum].split(" ")), " words"
        prob_of_s, total = get_sentence_probability(corpus_words,sentence)
        #print "The probabilities of all words in the sentence is : ", prob_of_s
        print "The total probability of the sentence is : ", total, "\n"
        enum += 1
    else:
        continue


(g) A word may have several part-of-speech tags, for example the word 'record' can be a noun or a verb. How many words do have more than one POS tag? What are the 10 most frequent combinations of POS tags?

In [ ]:
# Collect words (keys) with multiple pos (values with list longer than 1)
words_multiple_pos = []

for word, pos in word_pos_dic.items():
    if len(pos) > 1:
        words_multiple_pos.append(set(pos))
        
print "Words with more than one POS: ", len(words_multiple_pos)
print "POS tags combination examples: ", words_multiple_pos[0:10]

#frequency_combos = dict()

# compare sets and count equals
#for i in range(len(words_multiple_pos)):
#    for j in range(i+1, len(words_multiple_pos)):
#        # find a match
#        if words_multiple_pos[i] == words_multiple_pos[j]:
#            
#            if words_multiple_pos[i] in frequency_combos:
#                # increase count
#                frequency_combos[words_multiple_pos[i]] += 1
#            else:
#                # create new entry, start count at 2
#                frequency_combos[words_multiple_pos[i]] = 2
    
    
# 10 most frequent word-pos combinations
frequency_combos = Counter(word_pos_combo)

most_common_10_combo = frequency_combos.most_common(10)

print "\nThe 10 most common combinations: \n", most_common_10_combo

## Exercise 1.2

You are also provided with another file called **sec00.gold.tagged**. 
Section 00 of the Penn Treebank is typically used as development data.

(a) How many unseen words are present in the development data (i.e., words that have not occurred in the training data)?

In [ ]:
# Proccess development data
words_dev, pos_dev, word_pos_combo_dev, word_pos_dic_dev, distinct_words_dev, distinct_pos_dev, distinct_word_pos_combo_dev = read_corpus('sec00.gold.tagged')


# Elements in development data but not in training
unseen_words = distinct_words_dev.difference(distinct_words)
unseen_pos   = distinct_pos_dev.difference(distinct_pos)

print "\nNumber of unseen words: ", len(unseen_words), "\nFor example: ", list(unseen_words)[0:10]
print "Number of unseen pos: ", len(unseen_pos), "\nFor example: ", list(unseen_pos)[0:10]

(b) What are the three most common kind of unseen word (their POS tags)?

In [ ]:
# Count unseen 
unseen_word_list = []
unseen_pos_list = []

for word in words_dev:
    if (word in unseen_words):
        unseen_word_list.append(word)
        

for pos in pos_dev:
    if (pos in unseen_pos):
        unseen_pos_list.append(pos)

In [ ]:
# Most common unseen elements
corpus_word_dev = Counter(unseen_word_list)
corpus_pos_dev = Counter(unseen_pos_list)

most_common_3_word = corpus_word_dev.most_common(3)
most_common_3_pos = corpus_pos_dev.most_common(3)

print "The 3rd most common unseen words: \n", most_common_3_word
print "\nThe 3rd most common unseen pos: \n", most_common_3_pos

We can see that even with a large corpus as data, we still have many unseen words with significant count. However, this is not the case for the POS tags, since there is a finite manageable number of tags, it is likely that a large enough corpus will contain all.